$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\ketbra}[2]{\ket{#1}\bra{#2}}$$

# Import de Modules

In [1]:
from qutip import *

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from scipy.fft import fft, fftfreq

plt.switch_backend("module://ipykernel.pylab.backend_inline")

# 1. Système à deux niveaux

### Définition des vecteurs de base

In [2]:
etat_g = basis(2,0)  # |g>
etat_e = basis(2,1)  # |e>

gg = etat_g*etat_g.dag()  # |g><g|
ee = etat_e*etat_e.dag()  # |e><e|
ge = etat_g*etat_e.dag()  # |e><e|
eg = etat_e*etat_g.dag()  # |e><g|

### Définition 